In [9]:
import sys, os, math, glob, json
from subprocess import Popen, PIPE
from tqdm.auto import tqdm
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO

from prettytable import PrettyTable

from functools import reduce

pd.options.plotting.backend='matplotlib'
result_files = json.load(open("./results_json.json", 'r'))
result = result_files
for dataset in result_files.keys():
    for method in result_files[dataset].keys():
        R = pd.read_csv(result[dataset][method])
        query_sizes = [int(x.split('_')[1]) for x in R['query']]
        q_error_exp = [10**x for x in R['q_error']]
        R["query_size"] = query_sizes
        R['q_error_exp'] = q_error_exp
        R['bucket'] = [((int)(math.log10(x)))//2 for x in R['y_true']]
        R["method"] = method
        result[dataset][method] = R
    result[dataset] = reduce(lambda df1,df2: pd.merge(df1,df2,how='outer'), [d for d in result[dataset].values()])
# result = pd.read_csv("/Users/gratus/Desktop/Projects/SubgraphCounting/gcare/dataset/JS-approx1m/human_approx_result_2022-11-8-14-47.csv")

In [10]:
DATASET = "Human"

In [20]:
def to_boxplot_tex(DATASET):
    R = result[DATASET]
    grouped = R.groupby(['query_size', 'method'])['q_error_exp']
    STR = ""
    name_set = set()
    for (name, obj) in grouped:
        name_set.add(name[1])
        median = obj.quantile(q=0.5)
        q1 = obj.quantile(q=0.25)
        q3 = obj.quantile(q=0.75)
        maximum = obj.quantile(q=0.95)
        minimum = obj.quantile(q=0.05)
        S = f"""
            % {name[0], name[1][1:]} : #datapoints = {len(obj)}
            \\addplot+[fill,fill opacity=0.3,
                boxplot prepared={'{'}
                lower whisker={minimum},
                lower quartile={q1},
                median={median},
                upper quartile={q3},
                upper whisker={maximum}
            {'}'},
            ] coordinates {'{}'};
        """
        STR += S
    s = f"{[name[1:] for name in list(name_set)]}"[1:-1].replace("'", "")
    STR += f"\\legend{'{'}{s}{'}'}"
    print(STR)
    return STR
    # keys = grouped.groups
    # print(keys, grouped.indices)
    # median = grouped.quantile(q=0.5)
    # print(median)
    # return f"""median : {median}"""
to_boxplot_tex("Human");


            % (4, 'TreeSamp') : #datapoints = 200
            \addplot+[fill,fill opacity=0.3,
                boxplot prepared={
                lower whisker=0.9970111150161655,
                lower quartile=0.9986194028465246,
                median=1.0,
                upper quartile=1.0012095931129013,
                upper whisker=1.003690930920097
            },
            ] coordinates {};
        
            % (4, 'WanderJoin') : #datapoints = 200
            \addplot+[fill,fill opacity=0.3,
                boxplot prepared={
                lower whisker=6.097538841201184e-08,
                lower quartile=4.3521113662397734e-07,
                median=6.723596738695428e-06,
                upper quartile=0.01805550362672151,
                upper whisker=1.312845717551945
            },
            ] coordinates {};
        
            % (4, 'JSUB') : #datapoints = 200
            \addplot+[fill,fill opacity=0.3,
                boxplot prepared={
                lower

In [30]:
import plotly.express as px

sns.set(rc={'figure.figsize':(10, 8)})
plt.rcParams['text.usetex'] = True
g = px.box(
    result, x='query_size', y='q_error_exp',
    log_y=True,
    range_y=(1e-12, 1e12),
)

g.update_layout(yaxis_tickformat="~e")
# g.set_ylabel("Q-err")
# g.set_yscale("log")
# g.set_title("EU2005 - by query size")
# K = 4
# g.set_yticks([10**i for i in range(-2*K, 2*K+1, 2)])
# g.set_yticklabels([f"$10^{{{abs(i)}}}$" for i in range(-2*K, 2*K+1, 2)])
g